In [8]:
import sys
sys.path.append('../code/python')

import findspark
import pyspark

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import array, col
from pyspark.sql.types import StringType, FloatType, IntegerType
from pyspark.sql import functions as F

toDense = F.udf(lambda v: Vectors.dense(v.toArray()), VectorUDT())

import time

In [9]:
! rm -r metastore_db

rm: cannot remove ‘metastore_db’: No such file or directory


In [10]:
findspark.init('/users/snakanda/vista/spark-2.2.0-bin-hadoop2.7')

sc = pyspark.SparkContext(appName="vista")

In [11]:
from vista import Vista

In [12]:
sc

<SparkContext master=local[*] appName=vista>

In [13]:
def downstream_ml_func(features_df, results_dict, layer_index):
    lr = LogisticRegression(labelCol="label", featuresCol="features", elasticNetParam=0.5, regParam=0.05)
    train_df, test_df = features_df.randomSplit([0.6, 0.4], seed=2019)
    valid_df, test_df = test_df.randomSplit([0.5, 0.5], seed=2019)
    
    model = lr.fit(train_df)
    result = {}

    predictions = model.transform(train_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    result['train_acc'] = evaluator.evaluate(predictions)
    
    predictions = model.transform(valid_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                 metricName="accuracy")
    result['valid_acc'] = evaluator.evaluate(predictions)
    
    predictions = model.transform(test_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    result['test_acc'] = evaluator.evaluate(predictions)
    
    results_dict[layer_index] = result
    return results_dict

## Foods dataset

In [14]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'alexnet', 4, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/foods/foods_wo_interactions.csv',
                  '/users/snakanda/vista/data/foods/images',
                  20129, 130, mem_sys_rsv=3)

result = vista.run()
for k in [-1, -2, -3, -4]:
    print(k, result[k])
    
# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 133, 0.972117558402, deser


/usr/local/lib/python2.7/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


(-1, {'train_acc': 0.7921960072595281, 'test_acc': 0.8097278225806451, 'valid_acc': 0.7841049764793265})
(-2, {'train_acc': 0.7972281801682891, 'test_acc': 0.8079637096774194, 'valid_acc': 0.7897994553107205})
(-3, {'train_acc': 0.7972281801682891, 'test_acc': 0.8077116935483871, 'valid_acc': 0.7959891062144095})
(-4, {'train_acc': 0.7953308034977726, 'test_acc': 0.8059475806451613, 'valid_acc': 0.7974746224312949})


In [15]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'alexnet', 4, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/foods/foods_with_interactions.csv',
                  '/users/snakanda/vista/data/foods/images',
                  20129, 130, mem_sys_rsv=3)

result = vista.run()
for k in [-1, -2, -3, -4]:
    print(k, result[k])

# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 133, 0.972117558402, deser
(-1, {'train_acc': 0.8091899026563273, 'test_acc': 0.8230846774193549, 'valid_acc': 0.8051497895518692})
(-2, {'train_acc': 0.8095198812077214, 'test_acc': 0.8203125, 'valid_acc': 0.804159445407279})
(-3, {'train_acc': 0.8105923114997525, 'test_acc': 0.8175403225806451, 'valid_acc': 0.8078732359494925})
(-4, {'train_acc': 0.8069625474344168, 'test_acc': 0.8162802419354839, 'valid_acc': 0.8125773706362961})


In [16]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'resnet50', 5, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/foods/foods_wo_interactions.csv',
                  '/users/snakanda/vista/data/foods/images',
                  20129, 130, mem_sys_rsv=3)

result = vista.run()
for k in [-1, -2, -3, -4, -5]:
    print(k, result[k])
    
# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 140, 0.973514674302, deser
(-1, {'train_acc': 0.7987130836495627, 'test_acc': 0.8122479838709677, 'valid_acc': 0.8011884129735083})
(-2, {'train_acc': 0.8020128691635043, 'test_acc': 0.8084677419354839, 'valid_acc': 0.7994553107204754})
(-3, {'train_acc': 0.7986305890117142, 'test_acc': 0.8009072580645161, 'valid_acc': 0.7967318643228521})
(-4, {'train_acc': 0.7995380300280481, 'test_acc': 0.8051915322580645, 'valid_acc': 0.7922753156721961})
(-5, {'train_acc': 0.7950833195842271, 'test_acc': 0.8009072580645161, 'valid_acc': 0.8029215152265412})


In [17]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'resnet50', 5, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/foods/foods_with_interactions.csv',
                  '/users/snakanda/vista/data/foods/images',
                  20129, 130, mem_sys_rsv=3)


result = vista.run()
for k in [-1, -2, -3, -4, -5]:
    print(k, result[k])
    
# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 140, 0.973514674302, deser
(-1, {'train_acc': 0.8141395809272397, 'test_acc': 0.8251008064516129, 'valid_acc': 0.8138153008170339})
(-2, {'train_acc': 0.8135621184623, 'test_acc': 0.8170362903225806, 'valid_acc': 0.8130725427085912})
(-3, {'train_acc': 0.8099323543969642, 'test_acc': 0.8130040322580645, 'valid_acc': 0.8145580589254766})
(-4, {'train_acc': 0.8118297310674806, 'test_acc': 0.819304435483871, 'valid_acc': 0.8021787571180985})
(-5, {'train_acc': 0.8072100313479624, 'test_acc': 0.8107358870967742, 'valid_acc': 0.814310472889329})


### Foods Structured Features Only

In [18]:
def get_struct_df(sc, data_file_path):
    sql_context = pyspark.SQLContext(sc)
    struct_df = sql_context.read.format('csv').options(header='false', inferSchema='true').load(data_file_path)
    col_names = struct_df.schema.names
        
    struct_df = VectorAssembler(inputCols=col_names[1:-1], outputCol="features").transform(struct_df)
    struct_df = struct_df.withColumn("features", toDense("features"))
    struct_df = struct_df.withColumn("id", struct_df[col_names[0]].cast(StringType())) \
        .withColumn("label", struct_df[col_names[-1]].cast(IntegerType())) \
        .select("id", "features", "label")
    return struct_df

In [19]:
features_df = get_struct_df(sc, '/users/snakanda/vista/data/foods/foods_wo_interactions.csv')
print(downstream_ml_func(features_df, {}, 0))

{0: {'train_acc': 0.7952549051801987, 'test_acc': 0.8086670045615814, 'valid_acc': 0.795352323838081}}


In [20]:
features_df = get_struct_df(sc, '/users/snakanda/vista/data/foods/foods_with_interactions.csv')
print(downstream_ml_func(features_df, {}, 0))

{0: {'train_acc': 0.8068959759559192, 'test_acc': 0.811587147030185, 'valid_acc': 0.8162255750680188}}


## Amazon Dataset

In [21]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'alexnet', 4, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/amazon/amazon.csv',
                  '/users/snakanda/vista/data/amazon/images',
                  20129, 130, mem_sys_rsv=3)

result = vista.run()
for k in [-1, -2, -3, -4]:
    print(k, result[k])
    
# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 133, 0.972117558402, deser
(-1, {'train_acc': 0.6541223075018568, 'test_acc': 0.6593489780469342, 'valid_acc': 0.676530612244898})
(-2, {'train_acc': 0.6602294297268301, 'test_acc': 0.6578349735049205, 'valid_acc': 0.6739795918367347})
(-3, {'train_acc': 0.6603944870842617, 'test_acc': 0.6563209689629069, 'valid_acc': 0.6762755102040816})
(-4, {'train_acc': 0.6499133448873483, 'test_acc': 0.6515266212465304, 'valid_acc': 0.6691326530612245})


In [22]:
prev_time = time.time()
# mem_sys_rsv is an optional parameter. If not set a default value of 3 will be used.
vista = Vista("vista-example", 150, 8, 1, 'resnet50', 5, 0, downstream_ml_func,
                  '/users/snakanda/vista/data/amazon/amazon.csv',
                  '/users/snakanda/vista/data/amazon/images',
                  20129, 130, mem_sys_rsv=3)

result = vista.run()
for k in [-1, -2, -3, -4, -5]:
    print(k, result[k])
    
# print("Runtime (min): " + str((time.time()-prev_time)/60.0))

Vista Configs(join, cpu, np, heap, f_core, pers): b, 7, 238, 140, 0.973514674302, deser


Py4JJavaError: An error occurred while calling o9852.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3896.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3896.0 (TID 8343, localhost, executor driver): java.io.FileNotFoundException: /users/snakanda/vista/data/amazon/images/B000MD3NUE.jpg (Too many open files)
	at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileInputStream.<init>(RawLocalFileSystem.java:106)
	at org.apache.hadoop.fs.RawLocalFileSystem.open(RawLocalFileSystem.java:202)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:143)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:346)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:769)
	at org.apache.spark.input.PortableDataSt

In [ ]:
features_df = get_struct_df(sc, '/users/snakanda/vista/data/amazon/amazon.csv')
print(downstream_ml_func(features_df, {}, 0))

## Hog Features

In [ ]:
def get_hog_df(sc, data_file_path):
    sql_context = pyspark.SQLContext(sc)
    struct_df = sql_context.read.format('csv').options(header='false', inferSchema='true').load(data_file_path)
    
    col_names = struct_df.schema.names
            
    struct_df = VectorAssembler(inputCols=col_names[:-1], outputCol="features").transform(struct_df)
    struct_df = struct_df.withColumn("features", toDense("features"))
    struct_df = struct_df.withColumn("label", struct_df[col_names[-1]].cast(IntegerType())) \
        .select("features", "label")
    return struct_df

In [ ]:
hog_df = get_hog_df(sc, '/users/snakanda/vista/data/foods/foods_wo_interactions_with_hog.csv')
print(downstream_ml_func(hog_df, {}, 0))

In [ ]:
hog_df = get_hog_df(sc, '/users/snakanda/vista/data/foods/foods_with_interactions_with_hog.csv')
print(downstream_ml_func(hog_df, {}, 0))

In [ ]:
hog_df = get_hog_df(sc, '/users/snakanda/vista/data/amazon/amazon_with_hog.csv')
print(downstream_ml_func(hog_df, {}, 0))

In [7]:
sc.stop()